# 실습 04: OCR 엔진 성능 비교

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/master/notebooks/Lab04_OCR엔진비교.ipynb)

## 🎯 학습 목표
- 여러 OCR 엔진 비교 (Tesseract, PaddleOCR, EasyOCR)
- 정확도, 속도, 신뢰도 측정
- 최적 OCR 엔진 선택 방법

## ⏱️ 소요 시간: 30분
## 📊 난이도: ⭐⭐⭐☆☆


In [ ]:
# 한글 폰트 설치 및 설정 (Colab용)
!apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra
!fc-cache -fv

# matplotlib 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

print("✅ 한글 폰트 설정 완료!")


In [ ]:
# 환경 설정
!pip install -q paddlepaddle paddleocr easyocr python-Levenshtein pytesseract
!apt-get install -y tesseract-ocr tesseract-ocr-kor


In [ ]:
# 샘플 이미지 생성
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# 한글 폰트 로드
try:
    font = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 20)
except:
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 20)
    except:
        font = ImageFont.load_default()

# 샘플 이미지 생성
img = Image.new('RGB', (600, 200), color='white')
draw = ImageDraw.Draw(img)
draw.text((20, 20), "삼성전자 주식회사\n계약서\n2025년 1월 15일\n담당자: 홍길동\n연락처: 02-1234-5678", fill='black', font=font)
img.save('sample_document.jpg')

print("✅ 샘플 이미지 생성 완료!")
img


In [ ]:
# 3개 OCR 엔진으로 비교
import pytesseract
from paddleocr import PaddleOCR
import easyocr
import time
import cv2

img = cv2.imread('sample_document.jpg')
ground_truth = "삼성전자 주식회사 계약서 2025년 1월 15일 담당자: 홍길동 연락처: 02-1234-5678"

results = []

# 1. Tesseract
print("🔍 Tesseract 실행 중...")
start = time.time()
text_tess = pytesseract.image_to_string(img, lang='kor+eng').strip()
time_tess = time.time() - start

# 신뢰도 계산
data_tess = pytesseract.image_to_data(img, lang='kor+eng', output_type=pytesseract.Output.DICT)
conf_tess = [int(c) for c in data_tess['conf'] if int(c) > 0]
avg_conf_tess = sum(conf_tess) / len(conf_tess) if conf_tess else 0

results.append({
    'engine': 'Tesseract',
    'text': text_tess,
    'processing_time': time_tess,
    'confidence': avg_conf_tess,
})

print(f"   인식 결과: {text_tess}")
print(f"   소요 시간: {time_tess:.2f}초\n")

# 2. PaddleOCR (최신 API)
print("🔍 PaddleOCR 실행 중...")
start = time.time()
paddle_ocr = PaddleOCR(use_textline_orientation=True, lang='korean')
result_paddle = paddle_ocr.predict('sample_document.jpg')
paddle_result = result_paddle[0]
text_paddle = ' '.join(paddle_result['rec_texts'])
time_paddle = time.time() - start

# 신뢰도 계산
scores_paddle = paddle_result['rec_scores']
avg_conf_paddle = (sum(scores_paddle) / len(scores_paddle)) * 100 if scores_paddle else 0

results.append({
    'engine': 'PaddleOCR',
    'text': text_paddle,
    'processing_time': time_paddle,
    'confidence': avg_conf_paddle,
})

print(f"   인식 결과: {text_paddle}")
print(f"   소요 시간: {time_paddle:.2f}초\n")

# 3. EasyOCR
print("🔍 EasyOCR 실행 중...")
start = time.time()
easy_reader = easyocr.Reader(['ko', 'en'], gpu=False, verbose=False)
result_easy = easy_reader.readtext('sample_document.jpg')
text_easy = ' '.join([item[1] for item in result_easy])
time_easy = time.time() - start

# 신뢰도 계산
scores_easy = [item[2] for item in result_easy]
avg_conf_easy = (sum(scores_easy) / len(scores_easy)) * 100 if scores_easy else 0

results.append({
    'engine': 'EasyOCR',
    'text': text_easy,
    'processing_time': time_easy,
    'confidence': avg_conf_easy,
})

print(f"   인식 결과: {text_easy}")
print(f"   소요 시간: {time_easy:.2f}초\n")

print("="*70)
print("✅ 모든 OCR 엔진 비교 완료!")
print("="*70)


In [ ]:
# 결과 분석 및 시각화
import pandas as pd
import Levenshtein

# 정확도 계산 함수
def calc_accuracy(pred, truth):
    # 공백 제거 후 비교
    pred_clean = pred.replace(' ', '').replace('\n', '')
    truth_clean = truth.replace(' ', '').replace('\n', '')
    distance = Levenshtein.distance(pred_clean, truth_clean)
    return max(0, (1 - distance / max(len(pred_clean), len(truth_clean))) * 100)

# 각 엔진의 정확도 계산
for result in results:
    result['character_accuracy'] = calc_accuracy(result['text'], ground_truth)

# DataFrame 생성
df = pd.DataFrame(results)

print("\n" + "="*80)
print("📊 OCR 엔진 비교 결과")
print("="*80)
print(f"\n{'엔진':<15} {'정확도':<12} {'처리시간':<12} {'신뢰도':<12}")
print("-"*80)
for _, row in df.iterrows():
    print(f"{row['engine']:<15} {row['character_accuracy']:>6.2f}%     {row['processing_time']:>6.2f}초     {row['confidence']:>6.1f}%")
print("="*80)

# 최고 성능 엔진
best = df.loc[df['character_accuracy'].idxmax()]
print(f"\n🏆 최고 정확도: {best['engine']} ({best['character_accuracy']:.2f}%)")

# 가장 빠른 엔진
fastest = df.loc[df['processing_time'].idxmin()]
print(f"⚡ 최고 속도: {fastest['engine']} ({fastest['processing_time']:.2f}초)")

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 정확도 비교
axes[0].bar(df['engine'], df['character_accuracy'], color=['#3498db', '#2ecc71', '#e74c3c'])
axes[0].set_title('OCR 엔진별 정확도', fontsize=14, fontproperties=fontprop)
axes[0].set_ylabel('정확도 (%)', fontproperties=fontprop)
axes[0].set_ylim(0, 100)
for i, v in enumerate(df['character_accuracy']):
    axes[0].text(i, v + 2, f'{v:.1f}%', ha='center', fontproperties=fontprop)

# 처리 시간 비교
axes[1].bar(df['engine'], df['processing_time'], color=['#3498db', '#2ecc71', '#e74c3c'])
axes[1].set_title('OCR 엔진별 처리 시간', fontsize=14, fontproperties=fontprop)
axes[1].set_ylabel('처리 시간 (초)', fontproperties=fontprop)
for i, v in enumerate(df['processing_time']):
    axes[1].text(i, v + 0.1, f'{v:.2f}s', ha='center', fontproperties=fontprop)

plt.tight_layout()
plt.show()

print("\n✅ 분석 완료!")
